In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [ ]:
def main():
    pass

In [13]:
def query_builder(basic_search: str = None, includes_all: str = None, exact_sentence: str = None, includes_one: str = None, 
                  exclude: str = None, in_title: bool = None, language: str = None, interval: tuple[int, int] = None, 
                  only_science: bool = None, include_patents: bool = None, include_citations: bool = None, sort_by_date: bool = None, 
                  authors: list = None, sources: list = None)-> str:
    '''Create search queries for Google Scholar'''
    assert bool(includes_all or exact_sentence or includes_one) ^ bool(basic_search and not (authors or sources or in_title)), "Invalid search query." 
    return "https://scholar.google.com/scholar?" + \
        (f"&q={basic_search.replace(' ', '+')}" if basic_search else "") + \
        (f"&as_q={includes_all.replace(' ', '+')}" if includes_all else "") + \
        (f"&as_epq={exact_sentence.replace(' ', '+')}" if exact_sentence else "") + \
        (f"&as_oq={includes_one.replace(' ', '+')}" if includes_one else "") + \
        (f"&as_eq={exclude.replace(' ', '+')}" if exclude else "") + \
        (f"&as_occt=title" if in_title else "") + \
        (f"""&as_sauthors={'+'.join([f'"{a.replace(" ", "+")}"' for a in authors])}""" if authors else "") + \
        (f"""&as_publication={'+'.join([f'"{s.replace(" ", "+")}"' for s in sources])}""" if sources else "") + \
        (f"&lr=lang_{language}?hl={language}" if language else "") + \
        (((f"&as_ylo={interval[0]}" + \
            (f"&as_yhi={interval[1]}" if len(interval) == 2 else ""))) if interval else "") + \
        ("&as_rr=1" if only_science else "") + \
        ("&as_sdt=2007" if include_patents else "") + \
        ("&as_vis=1" if include_citations else "") + \
        ("&scisbd=1" if sort_by_date else "")

In [14]:
if __name__ == "__main__": 
    print(query_builder())

AssertionError: Invalid search query.